## Configs

In [45]:
offset = 6921
limit = 8000
period = 'max' # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max

## Download all NASDAQ traded symbols

In [23]:
import pandas as pd

data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
data_clean = data_clean.dropna(subset=['Symbol'])
symbols = data_clean['NASDAQ Symbol'].tolist()
etfs = data_clean[data_clean['ETF'] == 'Y']['NASDAQ Symbol'].tolist()
stocks = data_clean[data_clean['ETF'] == 'N']['NASDAQ Symbol'].tolist()
print('total number of symbols traded = {}'.format(len(symbols)))
print('total number of stocks traded = {}'.format(len(stocks)))
print('total number of etfs traded = {}'.format(len(etfs)))

total number of symbols traded = 11225
total number of stocks traded = 7941
total number of etfs traded = 3284


## Download Historic Stock Data

In [3]:
! pip install yfinance > /dev/null 2>&1
! mkdir stocks

In [4]:
import yfinance as yf
import os, contextlib

In [46]:
%%time

limit = limit if limit else len(stocks)
end = min(offset + limit, len(stocks))
is_valid = [False] * len(stocks)
# force silencing of verbose API
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        for i in range(offset, end):
            s = stocks[i]
            data = yf.download(s, period=period)
            if len(data.index) == 0:
                continue
        
            is_valid[i] = True
            data.to_csv('stocks/{}.csv'.format(s))

print('Total number of valid stocks downloaded = {}'.format(sum(is_valid)))


1 Failed download:
['TAP.A']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['TCOA=']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['TCOA+']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['TDS-U']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['TDS-V']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['TDW+']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['TFC-I']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['TFC-O']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['TFC-R']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['TGH-A']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['TGH-B']: Exception(

Total number of valid stocks downloaded = 944
CPU times: user 1min 21s, sys: 8.82 s, total: 1min 30s
Wall time: 17min 12s


In [47]:
valid_stock_data = pd.DataFrame(stocks)[is_valid]
valid_stock_data.to_csv('stocks_valid_meta.csv', index=False)